# Using the GA-D with USRCAT and Tanimoto
Matthew Roberts
## Introduction
### Using the Notebook
This notebook has been put together with all the necessary instructions to make use of the GA-D code (originally created by the Azpuru-Guzik group)[1]

For the set up, see either the original documentation for the GA-D (source: https://github.com/aspuru-guzik-group/GA) or see below. For the purposes of this notebook we will assume that Google Colab is being used, however if it is not, you may need to make some slight adjustments to methods discussed here to allow for them to work on your own terms.

### Setting Up

The requirements for the code are as follows:
- SELFIES (any version) : The code was run with v0.1.1 (which is the fastest), however, the code is compatible with any version.
- RDKit
- tensorboardX
- Pytorch v0.4.1
- Python 3.0 or up
- numpy

Run the code below to install a new conda environment with all the necessary requirements (if using Google Colab), or load an environment already available with these libraries installed.

In [ ]:
#New Conda install - Added to try to deal with Conda Problem
!wget -c https://repo.anaconda.com/miniconda/Miniconda3-py37_4.9.2-Linux-x86_64.sh
!chmod +x Miniconda3-py37_4.9.2-Linux-x86_64.sh	
!bash ./Miniconda3-py37_4.9.2-Linux-x86_64.sh -b -f -p /usr/local 

#Packages needed:
!conda install -y python=3.7 conda=4.9.2
!conda install -y -c rdkit rdkit conda=4.9.2
!pip install selfies
!pip install tensorboardX
!conda install -y pytorch=0.4.1 -c pytorch

Load dependancies and check it works (as a quick check, a benzene molecule should be drawn if rdkit has been installed correctly):

*Note: This only needs to be done on Colab*

In [ ]:
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdFMCS
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole

#An example to check it works - should be benzene
mol = Chem.MolFromSmiles('c1ccccc1')
mol

### Getting the GA-D code

We will be using a fresh copy of the GA-D from the source, however if the code has already been downloaded and is otherwise available, skip this step.

In [ ]:
!git clone --single-branch --branch paper_results https://github.com/akshat998/GA.git 

The cloned files should include several "Experiment" codes (4.1, 4.2, 4.3 etc.). Each of these has slightly different distinctions in terms of coding and use. Consult the original source or the literature [1] for more information on these.

Of interest here is the code Experiment 4.4, as this code encopases the ability to create new molecules through a fitness function with a particular objective in mind.

### Running the GA-D

To run any given experiment code, simply use the below command, replacing the file path for the file path of the intended target file. To start the GA-D the file activated should be a "core_GA.py" file.

The python file "core_GA.py" contains all the necessary commands to initiate the GA-D. The parameters which can be found in the bottom of the file control the number of generations, the population size of each generation, the properties to be calculated, and the nature of molecular representation (SMILES or SELFIES) among other things.

core_GA.py has to be edited to allow for the introduction of USRCAT and Tanimoto, hence use the replacement version available via GitHub.

link: https://github.com/Mattarian/GAD-USRCAT/tree/main/Exp%204%204%20with%20USRCAT%20and%20Tanimoto

In [ ]:
!python /content/GA/4.4/GA/core_GA.py

The other python files of note are the files "evolution_functions.py" and "generation_props.py".

#### evolution_functions
A length of code dedicated to the generation of new molecules via following 'evolution' paths. There is very little editing required for this code to work with new parameters, only a single line to allow for the introduction of new parameters into the outputted image files. See the edited versions on GitHub to find the edited line (it is marked with a #!#). 

The file "evolution_functions.py" will be called by the core_GA.py file when it has been initiated, hence besides editing nothing further is required of this file.

#### generation_props
A length of code dedicated to the calculation and determination of properties for generated and input molecules.

<span style="color:red"> This is the code which has been most heavily edited, hence it is suggested that users familiarise themselves with this code, if they are intending to further edit the code.

There are multiple important parts to the file "generation_props.py" which will be outlined below.

##### Molecule Property Calculations
The code to calculate properties is outlined at the very top of the file (just after the imports). There is one function to calculate each property, and they follow a fairly standardised layout. Should a new property be desired to be added, using one of the existing property functions as a layout and copying is the easiest way to do so (renaming relevent parts where appropriate). 

The general layout is given below, for a property X:

In [ ]:
def calc_prop_X(unseen_smile_ls, property_name, props_collect):
    '''Calculate X for each molecule in unseen_smile_ls, and record results
       in locked dictionary props_collect 
    '''
    for smile in unseen_smile_ls:
        mol, smi_canon, did_convert = evo.sanitize_smiles(smile)
        if did_convert:                                           
            #some calculation method here
            X = 1 + 2
            #Output results:
            props_collect[property_name][smile] = X
        else:
            raise Exception('Invalid smile encountered while atempting to calculate X')

The important part of this is that the output of the function is give through the line:

props_collect[property_name][smile] = ... etc

as this provides the code with the necessary outputted variable to be used by the remainder of the code.

From this initial calculation a number of different variables must be changed in the code to incorporate the molecular property, should a new molecular property be desired to be added to the existing code. ***In the available codes on GitHub the places where variables need to be changed or new variables for a newly added property are required to be added are labelled with the symbol #!#***

##### Fitness Function
The fitness function is visible at several points in the code, however to change the objective of the fitness function, J(m), the relevent code is found on line 238 in the code available in "Exp 4 4 with Tanimoto".

To fully calculate the fitness and objective the code requires normalised data. It is recommended to use the data intended to be used with the Discriminator to calculate the normalised results.

### Results of a Run
Results are saved into a number of files for each successive generation of molecules, as well as a final set of overall results. The files for each generation includes;
- a list of all molecule SMILES for that generation
- a list of all molecule properties (one file for each property) for each molecule
- a set of images for each generation, each image containing the molecular structure for every molecule in the generation as well as the properties for each molecule listed underneath
- a list of the best molecules in each generation 
- a list of the max/min values for each property

## References
1. A. Nigam, P. Friederich, M. Krenn, A. Aspuru-Guzik, 2019, arxiv preprint, arxiv: arXiv:1909.11655v4